In [3]:
# 필수 라이브러리 설치
!pip install roboflow singleton-decorator

import os
import sys
import cv2
import numpy as np
import torch
from singleton_decorator import singleton
import zipfile

# YOLOv5 레포지토리 클론 및 이동
if not os.path.exists('/content/yolov5'):
    !git clone https://github.com/ultralytics/yolov5 /content/yolov5
%cd /content/yolov5
!pip install -r requirements.txt

# Roboflow에서 데이터셋 다운로드
from roboflow import Roboflow

try:
    rf = Roboflow(api_key="gUNvjxIQ9rKYruOxpb3t")
    project = rf.workspace("large-benchmark-datasets").project("logistics-sz9jr")
    version = project.version(2)
    dataset = version.download("yolov5")

    # 압축 파일 경로
    zip_file_path = '/content/yolov5/Logistics-2/roboflow.zip'
    # 압축을 풀 폴더 경로
    extract_to_folder = '/content/yolov5/Logistics-2'
    # 압축 풀기
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_folder)
        print(f"Extracted all files to {extract_to_folder}")
except zipfile.BadZipFile:
    print("Error: The file is not a valid ZIP file or is corrupted.")

except Exception as e:
    print(f"Error downloading dataset from Roboflow: {e}")


@singleton
class ClsYoloGPU:
    def __init__(self, Param):
        self.Param = Param
        self.Param.LoadDbValue()
        self.__initCls()
        self.__deviceInfo()

    def __initCls(self):
        try:
            ModelFile = self.Param.model
            framework = os.path.join(os.getcwd(), 'yolov5')
            self.__model = torch.hub.load('/content/yolov5', 'custom', path=ModelFile, source='local')
            self.__model.conf = self.Param.confThresh
            self.__model.iou = self.Param.iouThresh
        except Exception as Ex:
            coName = sys._getframe().f_code.co_name
            print(f'{coName} = {Ex}')

    def __deviceInfo(self):
        device = (
            "cuda"
            if torch.cuda.is_available()
            else "mps"
            if torch.backends.mps.is_available()
            else "cpu"
        )
        print(f"Using {device} device")
        if device == "cuda":
            print(torch.cuda.get_device_name(0))
            print(torch.cuda.current_device())
            print(torch.cuda.device_count())

    def detectChar(self, img: np.ndarray) -> list:
        LsResult = []  # 초기화
        try:
            (h, w) = img.shape[:2]
            npCvs = np.copy(img)
            e1 = cv2.getTickCount()

            result = self.__model([img], size=1024)
            e2 = cv2.getTickCount()
            t = (e2 - e1) / cv2.getTickFrequency()
            print(f'Work Time = {t}sec')

            result.print()
            print(self.__model.conf)
            print(self.__model.iou)
            print(result.xyxy[0])
            print(result.pandas().xyxy[0])

            box = result.xyxy[0]
            print(f'yolov5 box = type {type(result.xyxy[0])}')

            resnp = result.xyxy[0].cpu().numpy()
            reInt = resnp.astype(np.uint64)

            LsOrg = resnp.tolist()
            LsResult = []

            for i, elem in enumerate(reInt.tolist()):
                TxDis = ''
                Score = LsOrg[i][4]
                tpLT = (elem[0], elem[1])
                tpRb = (elem[2], elem[3])
                TxDis = '{0}'.format(elem[5])

                tpData = (TxDis, Score, tpLT, tpRb)
                LsResult.append(tpData)

        except Exception as Ex:
            coName = sys._getframe().f_code.co_name
            print(f'{coName} = {Ex}')

        return LsResult

# ClsRecipe 클래스는 사용자 정의 클래스입니다. 이 예제에서는 더미 클래스로 대체합니다.
class ClsRecipe:
    def __init__(self, model, confThresh, iouThresh):
        self.model = model
        self.confThresh = confThresh
        self.iouThresh = iouThresh

    def LoadDbValue(self):
        pass  # 데이터베이스에서 값을 로드하는 로직을 추가합니다.

# 예제 파라미터 설정
param = ClsRecipe(model='yolov5s.pt', confThresh=0.25, iouThresh=0.45)
yolo = ClsYoloGPU(param)

# 이미지 예제
img_path = '/content/yolov5/data/images/zidane.jpg'
if not os.path.exists(img_path):
    !wget https://ultralytics.com/images/zidane.jpg -O /content/yolov5/data/images/zidane.jpg
img = cv2.imread(img_path)
results = yolo.detectChar(img)
print(results)

# 데이터 증강 설정 파일 수정 (hyp.scratch-low.yaml 파일 수정 예시)
hyp_path = 'data/hyps/hyp.scratch-low.yaml'
!sed -i 's/mosaic: 1.0/mosaic: 1.5/' {hyp_path}
!sed -i 's/mixup: 0.5/mixup: 1.0/' {hyp_path}
!sed -i 's/scale: 0.5/scale: 0.9/' {hyp_path}

# Roboflow에서 다운로드한 데이터셋 경로 확인
dataset_path = os.path.join(dataset.location, 'data.yaml')

# 학습 명령어 (전이 학습 및 적절한 배치 크기 사용)
!python train.py --data {dataset_path} --epochs 100 --weights yolov5s.pt --batch-size 16 --hyp {hyp_path} --img 640


/content/yolov5
loading Roboflow workspace...
loading Roboflow project...
Error downloading dataset from Roboflow: [Errno 2] No such file or directory: '/content/yolov5/Logistics-2/roboflow.zip'


YOLOv5 🚀 v7.0-350-g6096750f Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)

100%|██████████| 14.1M/14.1M [00:00<00:00, 196MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Using cuda device
NVIDIA A100-SXM4-40GB
0
1


WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/1: 720x1280 2 persons, 1 tie
Speed: 16.6ms pre-process, 118.8ms inference, 732.8ms NMS per image at shape (1, 3, 576, 1024)


Work Time = 0.86903395sec
0.25
0.45
tensor([[4.39435e+02, 4.35431e+02, 5.20919e+02, 7.18198e+02, 6.71138e-01, 2.70000e+01],
        [1.36378e+02, 2.04890e+02, 8.73237e+02, 7.14715e+02, 5.30856e-01, 0.00000e+00],
        [5.25618e+02, 3.17580e+01, 1.17449e+03, 7.20000e+02, 4.52984e-01, 0.00000e+00]], device='cuda:0')
         xmin        ymin         xmax        ymax  confidence  class    name
0  439.435303  435.431183   520.919312  718.197937    0.671138     27     tie
1  136.377594  204.890137   873.237183  714.714722    0.530856      0  person
2  525.617737   31.758003  1174.490234  720.000000    0.452984      0  person
yolov5 box = type <class 'torch.Tensor'>
[('27', 0.6711376309394836, (439, 435), (520, 718)), ('0', 0.5308561325073242, (136, 204), (873, 714)), ('0', 0.4529842138290405, (525, 31), (1174, 720))]
2024-07-29 08:22:59.026196: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN 